# Wind speed inversion from level-1 product 

In [ ]:
import xsarsea
from xsarsea import windspeed
import xarray as xr
import numpy as np
import holoviews as hv
hv.extension('bokeh')

import os,sys,re, cv2

In [ ]:
sarwing_luts_subset_path = xsarsea.utils.get_test_file('sarwing_luts_subset')
windspeed.register_all_sarwing_luts(sarwing_luts_subset_path)

In [ ]:
# optional debug messages
#import logging
#logging.basicConfig()
#logging.getLogger('xsarsea.windspeed').setLevel(logging.DEBUG) # or .setLevel(logging.INFO)

## Requirements for inversion

xsar is required

In [ ]:
import xsar

Getting metadata

In [ ]:
safe_path = xsarsea.get_test_file("S1A_EW_GRDM_1SDV_20230217T002336_20230217T002412_047268_05AC30_Z005.SAFE")
s1meta = xsar.Sentinel1Meta(safe_path)


In [ ]:
safe_path

In [ ]:
from xsarsea.utils import _load_config
config = _load_config()

land mask:
not applied yet 

getting associated ancillary data (ecmwf)

In [ ]:
s1meta.set_raster('ecmwf_0100_1h',config["path_ecmwf_0100_1h"])
import datetime
for ecmwf_name in ['ecmwf_0100_1h' ]:
    ecmwf_infos = s1meta.rasters.loc[ecmwf_name]
    ecmwf_file = ecmwf_infos['get_function'](ecmwf_infos['resource'], date=datetime.datetime.strptime(s1meta.start_date, '%Y-%m-%d %H:%M:%S.%f'))
    ecmwf_file = xsarsea.get_test_file(ecmwf_file[1].split('/')[-1],iszip=False)
    map_model = { '%s_%s' % (ecmwf_name, uv) : 'model_%s' % uv for uv in ['U10', 'V10'] }

In [ ]:
map_model

In [ ]:
s1meta.rasters.at["ecmwf_0100_1h","resource"] = ecmwf_file

Mapping model & adding ancillary wind 

In [ ]:
### Loading dataset & merging ancillary
xsar_obj_1000m = xsar.Sentinel1Dataset(s1meta, resolution='1000m')

In [ ]:
dataset_1000m = xsar_obj_1000m.datatree['measurement'].to_dataset()
dataset_1000m = dataset_1000m.rename(map_model)

creation of variables of interest for inversion 

here we could add a land/ice mask.

In [ ]:
### Variables of interest 
#xsar_obj_1000m.dataset['land_mask'].values = cv2.dilate(xsar_obj_1000m.dataset['land_mask'].values.astype('uint8'),np.ones((3,3),np.uint8),iterations = 3)
#xsar_obj_1000m.dataset['sigma0_ocean'] = xr.where(xsar_obj_1000m.dataset['land_mask'], np.nan, xsar_obj_1000m.dataset['sigma0'].compute()).transpose(*xsar_obj_1000m.dataset['sigma0'].dims)
#xsar_obj_1000m.dataset['sigma0_ocean'] = xr.where(xsar_obj_1000m.dataset['sigma0_ocean'] <= 0, 1e-15, xsar_obj_1000m.dataset['sigma0_ocean'])

In [ ]:
dataset_1000m['sigma0_ocean'] = xr.where(dataset_1000m['sigma0'] <= 0, 1e-15, xsar_obj_1000m.dataset['sigma0'])
dataset_1000m['ancillary_wind_direction'] = (90. - np.rad2deg(np.arctan2(dataset_1000m.model_V10, dataset_1000m.model_U10)) + 180) % 360
dataset_1000m['ancillary_wind_speed'] = np.sqrt(dataset_1000m['model_U10']**2+dataset_1000m['model_V10']**2)
dataset_1000m['ancillary_wind'] = dataset_1000m.ancillary_wind_speed * np.exp(1j * xsarsea.dir_geo_to_sample(dataset_1000m.ancillary_wind_direction, dataset_1000m.ground_heading)) # ref antenna

In [ ]:
hv.Image(dataset_1000m['sigma0_ocean'].sel(pol='VH')).opts(colorbar=True,cmap='binary',width=425, height=400, tools = ['hover'], title = "sigma0 VH")

## Inversion

### inversion parameters

In [ ]:
apply_flattening = True
GMF_VH_NAME = "gmf_s1_v2"

apply flattening or not

In [ ]:
nesz_cr = dataset_1000m.nesz.isel(pol=1) #(no_flattening)
if apply_flattening : 
    dataset_1000m=dataset_1000m.assign(nesz_VH_final=(['line','sample'],windspeed.nesz_flattening(nesz_cr, dataset_1000m.incidence)))
    dataset_1000m['nesz_VH_final'].attrs["comment"] = 'nesz has been flattened using windspeed.nesz_flattening'
else :
    dataset_1000m=dataset_1000m.assign(nesz_VH_final=(['line','sample'],nesz_cr.values))
    dataset_1000m['nesz_VH_final'].attrs["comment"] = 'nesz has not been flattened'

compute dsig_cr (mix between polarisations) using the last version : "gmf_s1_v2"

In [ ]:
dsig_cr = windspeed.get_dsig("gmf_s1_v2", dataset_1000m.incidence,dataset_1000m.sigma0_ocean.sel(pol='VH'),dataset_1000m.nesz_VH_final)

### get windspeed & direction in dfferent polarizations

copol and dual polarization

In [ ]:
wind_co, wind_dual = windspeed.invert_from_model(
        dataset_1000m.incidence,
        dataset_1000m.sigma0_ocean.isel(pol=0),
        dataset_1000m.sigma0_ocean.isel(pol=1),
        ancillary_wind=-np.conj(dataset_1000m['ancillary_wind']),
        dsig_cr = dsig_cr,
        model=('cmod5n','cmodms1ahw'))

dataset_1000m["windspeed_co"] = np.abs(wind_co)
dataset_1000m["windspeed_dual"] = np.abs(wind_dual)
dataset_1000m['winddir_co'] = (90 - (np.angle(-np.conj(wind_co), deg=True)) + dataset_1000m.ground_heading) % 360
dataset_1000m['winddir_dual'] = (90 - (np.angle(-np.conj(wind_dual), deg=True)) + dataset_1000m.ground_heading) % 360

cross polarization 

In [ ]:
windspeed_cr = windspeed.invert_from_model(
    dataset_1000m.incidence.values,
    dataset_1000m.sigma0_ocean.isel(pol=1).values,
    dsig_cr = dsig_cr.values,
    model=GMF_VH_NAME)

dataset_1000m = dataset_1000m.assign(
    windspeed_cross=(['line', 'sample'], windspeed_cr))

windspeeed illustration

In [ ]:
hv.Image(dataset_1000m.windspeed_co.compute(), label='wind speed co-pol').opts(cmap='jet',colorbar=True,clim=(0,80),height=400, width=425) + \
hv.Image(dataset_1000m.windspeed_cross, label='wind speed cr-pol').opts(cmap='jet',colorbar=True,clim=(0,80),height=400, width=425) + \
hv.Image(dataset_1000m.windspeed_dual.compute(), label='wind speed dual-pol').opts(cmap='jet',colorbar=True,clim=(0,80),height=400, width=425)

winddir illustration

In [ ]:
sub_ds = dataset_1000m.sel(pol='VV').isel(sample=slice(None, None, 10), line=slice(None, None, 10))

vectorfield = hv.VectorField(
    (
        sub_ds.sample, sub_ds.line,
        xsarsea.dir_geo_to_sample(sub_ds.winddir_dual,sub_ds.ground_heading),
        sub_ds.windspeed_dual
    )
)

hv.Image(dataset_1000m.windspeed_dual, kdims=['sample','line']).opts(title='speed and dir', clim=(0,50), cmap='jet') * vectorfield


In [ ]:
sub_ds = dataset_1000m.sel(pol='VV').isel(sample=slice(None, None, 10), line=slice(None, None, 10))

vectorfield = hv.VectorField(
    (
        sub_ds.sample, sub_ds.line,
        xsarsea.dir_geo_to_sample(sub_ds.ancillary_wind_direction,sub_ds.ground_heading),
        sub_ds.ancillary_wind_speed
    )
)

hv.Image(dataset_1000m.ancillary_wind_speed, kdims=['sample','line']).opts(title='ECMWF speed and dir', clim=(0,50), cmap='jet') * vectorfield


### save as a level-2 netcdf

delete useless variables

In [ ]:
# prepare dataset for netcdf export

dataset_1000m['sigma0_ocean_VV'] = dataset_1000m['sigma0_ocean'].sel(pol='VV')
dataset_1000m['sigma0_ocean_VH'] = dataset_1000m['sigma0_ocean'].sel(pol='VH')


black_list = ['model_U10', 'model_V10', 'digital_number', 'gamma0_raw', 'negz',
              'azimuth_time', 'slant_range_time', 'velocity', 'range_ground_spacing',
              'gamma0', 'time', 'sigma0', 'nesz', 'sigma0_raw', 'sigma0_ocean', 'altitude', 'elevation',
              'nd_co', 'nd_cr']

variables = list(set(dataset_1000m) - set(black_list))
dataset_1000m = dataset_1000m[variables]

remove complex

In [ ]:
del dataset_1000m['ancillary_wind']

In [ ]:
ds_1000 = dataset_1000m.compute()
ds_1000

In [ ]:
#ds_1000.to_netcdf("my_L2_product")